Getting all data types

In [134]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
from datetime import datetime, timedelta

exchange_mapping = {
    'NMS': 'NASDAQ',
    'NYQ': 'NYSE',
}

sector_indices = pd.read_csv('sectors.csv', index_col='sector')

def fetch_stock_data(ticker_symbol, start_date, length):
    try:
        start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
        end_date_obj = start_date_obj + timedelta(days=length)
        end_date = end_date_obj.strftime('%Y-%m-%d')
        
        ticker = yf.Ticker(ticker_symbol)
        daily_data = ticker.history(start=start_date, end=end_date)
        
        financials = ticker.quarterly_financials.T
        financials.index = pd.to_datetime(financials.index)
        financials = financials.sort_index()

        if financials.empty:
            raise ValueError(f"No financial data available for {ticker_symbol}")
        
        float_shares = ticker.info.get('floatShares', 'N/A')
        sector = ticker.info.get('sector', 'N/A')
        exchange_code = ticker.info.get('exchange', 'N/A')
        exchange = exchange_mapping.get(exchange_code, exchange_code)
        
        earnings_dates = ticker.earnings_dates.sort_index(ascending=False).head(4) if not ticker.earnings_dates.empty else None
        sector_index_ticker = sector_indices.loc[sector, 'index'] if sector in sector_indices.index else None
        sector_data = yf.Ticker(sector_index_ticker).history(start=start_date, end=end_date) if sector_index_ticker else None
        
        return daily_data, financials, float_shares, sector_data, sector, exchange, earnings_dates
    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {e}")
        return None, None, None, None, None, None, None

def plot_stock_data(ticker_symbol, daily_data, financial_data, float_shares, sector_data, sector, exchange, earnings_dates):
    plt.style.use('dark_background')
    fig, axes = plt.subplots(6, 1, figsize=(24, 36), gridspec_kw={'height_ratios': [2.5, 0.6, 0.6, 0.6, 0.6, 0.6]})
    ax1, ax2, ax3, ax4, ax5, ax6 = axes

    def format_date_axis(ax):
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.set_facecolor('black')

    if not daily_data.empty:
        daily_data.index = daily_data.index.tz_localize(None)
        
        daily_data['Date'] = mdates.date2num(daily_data.index.to_pydatetime())
        candlestick_ohlc(ax1, daily_data[['Date', 'Open', 'High', 'Low', 'Close']].values, width=0.6, colorup='green', colordown='red')
        ax1.set_title(f'{ticker_symbol}\nSector: {sector}\nExchange: {exchange}', fontsize=30)
        ax1.set_ylabel('Price', fontsize=20)

        if sector_data is not None:
            sector_data['Date'] = mdates.date2num(sector_data.index.to_pydatetime())
            ax6.plot(sector_data['Date'], sector_data['Close'], color='white', linewidth=2)
            ax6.set_ylabel('Sector Price', fontsize=20)
        else:
            ax6.text(0.5, 0.5, f"No sector data available for {sector}", transform=ax6.transAxes, fontsize=20, ha='center', va='center', color='red')

        ax2.bar(daily_data['Date'], daily_data['Volume'], color='white', width=0.6)
        ax2.set_ylabel('Volume', fontsize=20)

        ax3.plot(daily_data['Date'], [float_shares] * len(daily_data), color='white', linewidth=2)
        ax3.set_ylabel('Float Shares', fontsize=20)

        start_date = daily_data.index.min()
        end_date = daily_data.index.max()

        financial_data_numeric = financial_data.select_dtypes(include=['number'])
        financial_data_numeric = financial_data_numeric.reindex(pd.date_range(start=start_date, end=end_date, freq='D')).interpolate(method='time')
        financial_data.update(financial_data_numeric)
        financial_data = financial_data.loc[start_date:end_date]

        if not financial_data.empty and 'Basic EPS' in financial_data.columns:
            financial_data['Date'] = mdates.date2num(financial_data.index.to_pydatetime())
            ax4.step(financial_data['Date'], financial_data['Basic EPS'].ffill(), where='mid', color='white')
            ax4.set_ylabel('EPS', fontsize=20)
        else:
            ax4.text(0.5, 0.5, "EPS data not available", transform=ax4.transAxes, fontsize=20, ha='center', va='center', color='red')

        if not financial_data.empty and 'Total Revenue' in financial_data.columns:
            ax5.step(financial_data['Date'], financial_data['Total Revenue'].ffill()/1e9, where='mid', color='white')
            ax5.set_ylabel('Revenue (B)', fontsize=20)
        else:
            ax5.text(0.5, 0.5, "Revenue data not available", transform=ax5.transAxes, fontsize=20, ha='center', va='center', color='red')

        for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
            format_date_axis(ax)
    else:
        for ax in axes:
            ax.text(0.5, 0.5, f"Failed to fetch data for {ticker_symbol}", transform=ax.transAxes, fontsize=20, ha='center', va='center', color='red')

    plt.tight_layout(pad=4.0)
    plt.subplots_adjust(top=0.95)
    plt.savefig(f'{ticker_symbol}_data.png', dpi=300, bbox_inches='tight')
    plt.close()

def get_stock_data(ticker_symbol, start_date, length):
    data = fetch_stock_data(ticker_symbol, start_date, length)
    if data[0] is not None:
        plot_stock_data(ticker_symbol, *data)
    else:
        print(f"Failed to fetch data for {ticker_symbol}")

ticker_symbol = 'MSFT'
start_date = '2005-01-01'
length = 150
get_stock_data(ticker_symbol, start_date, length)


Error fetching data for MSFT: Have not implemented fetching 'earnings' from Yahoo API
Failed to fetch data for MSFT
